<a href="https://colab.research.google.com/github/SandeshBashyal/Pytorch_Offical_Tutorials/blob/main/Optimizing_Model_Parameter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Optimizing Model Parameter

Training a model is an iterative process; in each iteration the model makes a guess about the output, calculates the error in its guess (loss), collects the derivatives of the error with respect to its parameters (as we saw in the previous section), and optimizes these parameters using gradient descent.

## Prerequisite Code

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

100%|██████████| 26421880/26421880 [00:01<00:00, 17106283.29it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 310122.18it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 5493285.82it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 21700780.90it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



## Hyperparameters

1. `Number of Epochs` - the number times to iterate over the dataset

    epochs = 5

2. `Batch Size` - the number of data samples propagated through the network before the parameters are updated

    batch_size = 64

3. `Learning Rate` - how much to update models parameters at each batch/epoch. Smaller values yield slow learning speed, while large values may result in unpredictable behavior during training.

    learning_rate = 1e-3

## Optimization Loop

Each epoch consists of two main parts:

1. `The Train Loop` - iterate over the training dataset and try to converge to optimal parameters.

2. `The Validation/Test Loop` - iterate over the test dataset to check if model performance is improving.

### Loss Function

Loss function measures the degree of dissimilarity of obtained result to the target value, and it is the loss function that we want to minimize during training. To calculate the loss we make a prediction using the inputs of our given data sample and compare it against the true data label value.

Loss function: https://pytorch.org/docs/stable/nn.html#loss-functions

 #Initialize the loss function

loss_fn = nn.CrossEntropyLoss()

### Optimizer

Optimization is the process of adjusting model parameters to reduce model error in each training step. Optimization algorithms define how this process is performed.

Optimizers: https://pytorch.org/docs/stable/optim.html

`optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)`

Inside the training loop, optimization happens in three steps:

1. Call `optimizer.zero_grad()` to reset the gradients of model parameters. Gradients by default add up; to prevent double-counting, we explicitly zero them at each iteration.

2. Backpropagate the prediction loss with a call to `loss.backward()`. PyTorch deposits the gradients of the loss w.r.t. each parameter.

3. Once we have our gradients, we call `optimizer.step()` to adjust the parameters by the gradients collected in the backward pass.

## Full Implementation

We define train_loop that loops over our optimization code, and test_loop that evaluates the model’s performance against our test data.

In [2]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [3]:
loss_fn = nn.CrossEntropyLoss()
learning_rate = 1e-3
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.296728  [   64/60000]
loss: 2.288447  [ 6464/60000]
loss: 2.273647  [12864/60000]
loss: 2.268656  [19264/60000]
loss: 2.250885  [25664/60000]
loss: 2.221303  [32064/60000]
loss: 2.222834  [38464/60000]
loss: 2.190651  [44864/60000]
loss: 2.183578  [51264/60000]
loss: 2.154458  [57664/60000]
Test Error: 
 Accuracy: 55.2%, Avg loss: 2.148964 

Epoch 2
-------------------------------
loss: 2.155740  [   64/60000]
loss: 2.150508  [ 6464/60000]
loss: 2.102561  [12864/60000]
loss: 2.112065  [19264/60000]
loss: 2.061265  [25664/60000]
loss: 2.001816  [32064/60000]
loss: 2.017304  [38464/60000]
loss: 1.945371  [44864/60000]
loss: 1.946557  [51264/60000]
loss: 1.867633  [57664/60000]
Test Error: 
 Accuracy: 56.1%, Avg loss: 1.874159 

Epoch 3
-------------------------------
loss: 1.910181  [   64/60000]
loss: 1.883462  [ 6464/60000]
loss: 1.776849  [12864/60000]
loss: 1.800457  [19264/60000]
loss: 1.694835  [25664/60000]
loss: 1.650718  [32064/600